# Machine Learning Engineer Nanodegree
## Deep Learning
## Project: Build a Digit Recognition Program

In this notebook, a template is provided for you to implement your functionality in stages which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission, if necessary. Sections that begin with **'Implementation'** in the header indicate where you should begin your implementation for your project. Note that some sections of implementation are optional, and will be marked with **'Optional'** in the header.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

----
## Step 1: Design and Test a Model Architecture
Design and implement a deep learning model that learns to recognize sequences of digits. Train the model using synthetic data generated by concatenating character images from [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) or [MNIST](http://yann.lecun.com/exdb/mnist/). To produce a synthetic sequence of digits for testing, you can for example limit yourself to sequences up to five digits, and use five classifiers on top of your deep network. You would have to incorporate an additional ‘blank’ character to account for shorter number sequences.

There are various aspects to consider when thinking about this problem:
- Your model can be derived from a deep neural net or a convolutional network.
- You could experiment sharing or not the weights between the softmax classifiers.
- You can also use a recurrent network in your deep neural net to replace the classification layers and directly emit the sequence of digits one-at-a-time.

Here is an example of a [published baseline model on this problem](http://static.googleusercontent.com/media/research.google.com/en//pubs/archive/42241.pdf). ([video](https://www.youtube.com/watch?v=vGPI_JvLoN0))

### Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [4]:
# 1. Import section
from __future__ import print_function
import numpy as np
from keras.datasets import mnist
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Convolution2D, MaxPooling2D
from keras.models import Model
from keras.utils import np_utils
from keras import backend as K
from mnist_data_preps import generate_sequences
from matplotlib import pyplot as plt
import matplotlib as mpl
import pickle
import tensorflow as tf
import datetime
import os
tf.python.control_flow_ops = tf # to overcome some compatibility issues between keras and tf

In [9]:
# 2. Method to generate a synthetic sequences out of MNIST dataset

(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

empty_space = -1  # empty spaces will be encoded as "-1"
nb_classes = 11  # include 10 digits and "-1" which designates empty space in digit sequence.
max_length = 5 # max number of digits in sequence


def generate_sequences(images, labels, out_size, max_length=max_length):
    """
    This function is used to generate a sequences of digits out of individual images.
    The function assumes uniform distribution of sequence legths.

    Inputs:
        images - individual digits as np.arrays
        labels  - labels for each individual digit
        out_size - number of output images (sequences of individual images)
        max_lenght - max number of digits in sequence.

    Outputs:
        sequences - numpy arrays with sequences. All images has the same size (image_size * sequence_length)
        sequence_labels - 2D numpy array (out_size * sequence_length). "-1" designates empty space for digit.
        sequence_length - length of the sequence (include only real digits)
    """
    # seq_length = out_size // max_length  # the number of generated synthetic image for each length.

    sequences = np.empty(shape=(out_size, 28, 28 * max_length), dtype=np.float32)
    sequence_labels = np.empty(shape=(out_size, max_length, nb_classes), dtype=np.int32)
    sequence_lengths = np.empty(shape=(out_size, (max_length+1)), dtype=np.int32)

    index = 0

    for _ in xrange(out_size):

        # get random number of digits for given sequence
        real_digits = np.random.choice(max_length) + 1 # TODO: this implies that each sequence will have betweeen 1 and 5 real digits (by default)
        noisy_digits = max_length - real_digits

        sequences[index], tmp_labels = \
            get_sequence_image(images, labels, real_digits, noisy_digits)

        sequence_labels[index] = np_utils.to_categorical(tmp_labels, nb_classes) # 1-hot encoding

        tmp_length = np.zeros((max_length+1))
        tmp_length[real_digits] = 1
        sequence_lengths[index] = tmp_length # 1 hot encoding

        index += 1

    return sequences, sequence_labels, sequence_lengths

def get_sequence_image(x, y, real_digits, noisy_digits):
    length = (real_digits + noisy_digits)
    output_image = np.empty(shape=(28, 28 * length), dtype=np.float32)
    output_label = np.empty(shape=(length), dtype=np.int32)

    for i in xrange(real_digits):
        rand_index = np.random.choice(np.shape(x)[0])
        output_image[:, 28 * i:(28 * (i + 1))] = x[rand_index, :, :]
        output_label[i] = y[rand_index]

    for j in xrange((real_digits), (real_digits + noisy_digits)):
        output_image[:, 28 * j:(28 * (j + 1))] = get_rand_image()
        output_label[j] = empty_space

    return output_image, output_label


def get_rand_image(x_dim=28, y_dim=28):
    return np.random.rand(x_dim, y_dim)

In [10]:
# 3. Method to prepare data

def prepare_MNIST_data():
    # the data, shuffled and split between train and test sets
    (train_images, train_labels), (test_images, test_labels) = mnist.load_data()

    X_train_orig, Y_train, Y_train_length = generate_sequences(train_images, train_labels, out_size=np.shape(train_images)[0],
                                                          max_length=sequence_length)
    X_test_orig, Y_test, Y_test_length = generate_sequences(test_images, test_labels, out_size=np.shape(test_images)[0],
                                                       max_length=sequence_length)

    if K.image_dim_ordering() == 'th':
        X_train = X_train_orig.reshape(X_train_orig.shape[0], 1, img_rows, img_cols)
        X_test = X_test_orig.reshape(X_test_orig.shape[0], 1, img_rows, img_cols)
        input_shape = (1, img_rows, img_cols)
    else:
        X_train = X_train_orig.reshape(X_train_orig.shape[0], img_rows, img_cols, 1)
        X_test = X_test_orig.reshape(X_test_orig.shape[0], img_rows, img_cols, 1)
        input_shape = (img_rows, img_cols, 1)

    # Classifiers' inputs and targets
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')

    # 1. Length classifier
    length_cls_train = Y_train_length
    length_cls_test = Y_test_length

    # 2. First digit classifier
    first_cls_train = Y_train[:, 0, :]
    first_cls_test = Y_test[:, 0, :]

    # 3.Second digit classifier
    second_cls_train = Y_train[:, 1, :]
    second_cls_test = Y_test[:, 1, :]

    # 4. Third digit classifier
    third_cls_train = Y_train[:, 2, :]
    third_cls_test = Y_test[:, 2, :]

    # 5. Forth digit classifier
    forth_cls_train = Y_train[:, 3, :]
    forth_cls_test = Y_test[:, 3, :]

    # 6. First digit classifier
    fifth_cls_train = Y_train[:, 4, :]
    fifth_cls_test = Y_test[:, 4, :]

    train_targets = [length_cls_train, first_cls_train, second_cls_train, third_cls_train, forth_cls_train, fifth_cls_train]
    test_targets = [length_cls_test, first_cls_test, second_cls_test, third_cls_test, forth_cls_test, fifth_cls_test]

    return X_train, train_targets, X_test, test_targets, input_shape

In [23]:
# 4. Method to create CNN model 
def synth_MNIST_model(cls_weights):
    # Shared feature layers which will be used for multiple classifiers.
    main_input = Input(shape=input_shape, dtype='float32', name='main_input')
    shared = Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                           border_mode='valid', input_shape=input_shape)(main_input)
    shared = Activation('relu')(shared)
    shared = Convolution2D(nb_filters, kernel_size[0], kernel_size[1])(shared)
    shared = Activation('relu')(shared)
    shared = MaxPooling2D(pool_size=pool_size)(shared)

    # # TODO: note that this is an extra layer, Not all weights will work with it.
    # shared = Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
    #                        border_mode='valid', input_shape=input_shape)(main_input)
    # shared = Activation('relu')(shared)
    # shared = MaxPooling2D(pool_size=pool_size)(shared)

    shared = Dropout(0.25)(shared)
    shared = Flatten()(shared)
    shared = Dense(128)(shared)
    shared = Activation('relu')(shared)
    shared = Dropout(0.5)(shared)

    # 6 different classifiers.
    # 1. Length classifier
    length_cls = Dense((sequence_length + 1))(shared)  # to account for sequence length + 1
    length_cls = Activation('softmax', name="length")(length_cls)

    # 2. First digit classifier
    first_cls = Dense(nb_classes)(shared)
    first_cls = Activation('softmax', name="first_digit")(first_cls)

    # 3. Second digit classifier
    second_cls = Dense(nb_classes)(shared)
    second_cls = Activation('softmax', name="second_digit")(second_cls)

    # 4. Third digit classifier
    third_cls = Dense(nb_classes)(shared)
    third_cls = Activation('softmax', name="third_digit")(third_cls)

    # 5. Forth digit classifier
    forth_cls = Dense(nb_classes)(shared)
    forth_cls = Activation('softmax', name="forth_digit")(forth_cls)

    # 6. Fifth digit classifier
    fifth_cls = Dense(nb_classes)(shared)
    fifth_cls = Activation('softmax', name="fifth_digit")(fifth_cls)

    # model compilation and training
    model = Model(input=[main_input], output=[length_cls, first_cls, second_cls, third_cls, forth_cls, fifth_cls])

    model.compile(loss={"length":"categorical_crossentropy", "first_digit":"categorical_crossentropy", 
                        "second_digit":"categorical_crossentropy", "third_digit":"categorical_crossentropy",
                        "forth_digit":"categorical_crossentropy", "fifth_digit":"categorical_crossentropy"},
                  optimizer='adadelta',
                  metrics=['accuracy'], loss_weights=cls_weights)

    return model

def train_model(model, train_inputs=None, train_targets=None, test_inputs=None, test_targets=None, use_trained=False):
    if use_trained:
        model = model
        model.load_weights(pretrained_weights)
    else:
        #create appropriate directory to store  weights and logs.
        directory = "output/" + str(datetime.datetime.now())
        if not os.path.exists(directory):
            os.makedirs(directory)

        checkpointer = ModelCheckpoint(filepath=directory+"/weights.hdf5", verbose=1, save_best_only=True)
        tensorboard = TensorBoard(log_dir=directory, histogram_freq=0, write_graph=True, write_images=False)
        model.fit([train_inputs], train_targets, batch_size=batch_size, nb_epoch=nb_epoch,
                  verbose=1, validation_data=([test_inputs], test_targets), callbacks=[checkpointer, tensorboard])

    return model

def evaluate_model(model, eval_inputs, eval_targets, verbose=1):
    score = model.evaluate(eval_inputs, eval_targets, verbose=verbose)
    for idx in xrange(len(score)):
        print(model.metrics_names[idx]+": "+str(score[idx]))

In [27]:
# 5. Training and evalution of the model

# General configuration
batch_size = 128
nb_classes = 11  # include 10 digits and "-1" which designates empty space in digit sequence.
nb_epoch = 12
sequence_length = 5

# input image dimensions
img_rows, img_cols = 28, (28 * sequence_length) # this is for MNIST
input_shape = (img_rows, img_cols, img_depth)
# number of convolutional filters to use
nb_filters = 32 # used to be 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)
pretrained_weights = "tmp/best_weights.hdf5"
cls_weights = [1., 1., 1., 1., 1., 1.]


X_train, train_targets, X_test, test_targets, input_shape = prepare_MNIST_data()

# Initiation and training of the model
new_model = synth_MNIST_model(cls_weights)
trained_model = train_model(model=new_model, train_inputs=X_train,
                            train_targets=train_targets, test_inputs=X_test,test_targets=test_targets, use_trained=False)

evaluate_model(trained_model,X_test,test_targets)

### Sampling predictions
fig = plt.figure()

for i in range(1,10):
    random_index = np.random.choice(10000)
    prediction = predict_sequence(trained_model, pred_input=X_test[np.newaxis, random_index])

    # Create subplot
    a = fig.add_subplot(3, 3, i)
    imgplot = plt.imshow(X_test_orig[random_index]) #TODO handle this case
    a.set_title('Predicted:'+prediction)
plt.show()

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 753s - loss: 7.8325 - length_loss: 0.6487 - first_digit_loss: 1.8769 - second_digit_loss: 1.7527 - third_digit_loss: 1.5216 - forth_digit_loss: 1.2049 - fifth_digit_loss: 0.8277 - length_acc: 0.8377 - first_digit_acc: 0.5627 - second_digit_acc: 0.5877 - third_digit_acc: 0.6346 - forth_digit_acc: 0.7070 - fifth_digit_acc: 0.8034 - val_loss: 1.1556 - val_length_loss: 0.0120 - val_first_digit_loss: 0.2383 - val_second_digit_loss: 0.2553 - val_third_digit_loss: 0.2262 - val_forth_digit_loss: 0.2354 - val_fifth_digit_loss: 0.1884 - val_length_acc: 0.9999 - val_first_digit_acc: 0.9441 - val_second_digit_acc: 0.9359 - val_third_digit_acc: 0.9433 - val_forth_digit_acc: 0.9390 - val_fifth_digit_acc: 0.9463
Epoch 2/12
60000/60000 [==============================] - 889s - loss: 2.4703 - length_loss: 0.1162 - first_digit_loss: 0.5715 - second_digit_loss: 0.5507 - third_digit_loss: 0.5031 - f

KeyboardInterrupt: 

### Question 1
_What approach did you take in coming up with a solution to this problem?_

**Answer:** 

THere are certain priciple solutions I have made:
1. **Data.** I created a synthetic data set for sequence recognition based on famouse MNIST dataset. For this I created the 
2. **Model** I used a small CNN model for digit recognition. The trick here is that I used Keras innate functionality to share certain layers between 
3. **Prediction approach** 


### Question 2
_What does your final architecture look like? (Type of model, layers, sizes, connectivity, etc.)_

**Answer:**
- For initial model as proof of concept I used quite small model based on the 2 convolution layers. 
- Activation function. I used Rectified Linear Unit, as it's believe to demostrate better outcomes than more classical sigmoids and tanh.
- Regularization. 




### Question 3
_How did you train your model? How did you generate your synthetic dataset?_ Include examples of images from the synthetic data you constructed.

**Answer:**


----
## Step 2: Train a Model on a Realistic Dataset
Once you have settled on a good architecture, you can train your model on real data. In particular, the [Street View House Numbers (SVHN)](http://ufldl.stanford.edu/housenumbers/) dataset is a good large-scale dataset collected from house numbers in Google Street View. Training on this more challenging dataset, where the digits are not neatly lined-up and have various skews, fonts and colors, likely means you have to do some hyperparameter exploration to perform well.

### Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:


### Your code implementation goes here.
### Feel free to use as many code cells as needed.



### Question 4
_Describe how you set up the training and testing data for your model. How does the model perform on a realistic dataset?_

**Answer:**

### Question 5
_What changes did you have to make, if any, to achieve "good" results? Were there any options you explored that made the results worse?_

**Answer:**

### Question 6
_What were your initial and final results with testing on a realistic dataset? Do you believe your model is doing a good enough job at classifying numbers correctly?_

**Answer:**

----
## Step 3: Test a Model on Newly-Captured Images

Take several pictures of numbers that you find around you (at least five), and run them through your classifier on your computer to produce example results. Alternatively (optionally), you can try using OpenCV / SimpleCV / Pygame to capture live images from a webcam and run those through your classifier.

### Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:


### Your code implementation goes here.
### Feel free to use as many code cells as needed.



### Question 7
_Choose five candidate images of numbers you took from around you and provide them in the report. Are there any particular qualities of the image(s) that might make classification difficult?_

**Answer:**

### Question 8
_Is your model able to perform equally well on captured pictures or a live camera stream when compared to testing on the realistic dataset?_

**Answer:**

### Optional: Question 9
_If necessary, provide documentation for how an interface was built for your model to load and classify newly-acquired images._

**Answer:** Leave blank if you did not complete this part.

----
### Step 4: Explore an Improvement for a Model

There are many things you can do once you have the basic classifier in place. One example would be to also localize where the numbers are on the image. The SVHN dataset provides bounding boxes that you can tune to train a localizer. Train a regression loss to the coordinates of the bounding box, and then test it. 

### Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:


### Your code implementation goes here.
### Feel free to use as many code cells as needed.



### Question 10
_How well does your model localize numbers on the testing set from the realistic dataset? Do your classification results change at all with localization included?_

**Answer:**

### Question 11
_Test the localization function on the images you captured in **Step 3**. Does the model accurately calculate a bounding box for the numbers in the images you found? If you did not use a graphical interface, you may need to investigate the bounding boxes by hand._ Provide an example of the localization created on a captured image.

**Answer:**

----
## Optional Step 5: Build an Application or Program for a Model
Take your project one step further. If you're interested, look to build an Android application or even a more robust Python program that can interface with input images and display the classified numbers and even the bounding boxes. You can for example try to build an augmented reality app by overlaying your answer on the image like the [Word Lens](https://en.wikipedia.org/wiki/Word_Lens) app does.

Loading a TensorFlow model into a camera app on Android is demonstrated in the [TensorFlow Android demo app](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/examples/android), which you can simply modify.

If you decide to explore this optional route, be sure to document your interface and implementation, along with significant results you find. You can see the additional rubric items that you could be evaluated on by [following this link](https://review.udacity.com/#!/rubrics/413/view).

### Optional Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:


### Your optional code implementation goes here.
### Feel free to use as many code cells as needed.



### Documentation
Provide additional documentation sufficient for detailing the implementation of the Android application or Python program for visualizing the classification of numbers in images. It should be clear how the program or application works. Demonstrations should be provided. 

_Write your documentation here._

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.